# Ensemble learning

In [0]:
!pip install --upgrade scikit-learn
!pip install --upgrade sklearn
!pip install --upgrade scipy
!pip install lightgbm
!pip install xgboost
!pip install unidecode
!pip install spacy

Requirement already up-to-date: scikit-learn in c:\programdata\anaconda3\lib\site-packages (0.22.2.post1)
Requirement already up-to-date: sklearn in c:\programdata\anaconda3\lib\site-packages (0.0)
Requirement already up-to-date: scipy in c:\programdata\anaconda3\lib\site-packages (1.4.1)


In [0]:
import pandas as pd
import numpy as np
import scipy
from PIL import Image
import os, sys
from glob import glob
import gc
import unidecode
import zipfile
import cv2
from tqdm import tqdm_notebook,trange
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
! python -m spacy download fr_core_news_sm
! python -m spacy download en_core_web_sm
! pip install catboost

In [0]:
from catboost import CatBoostClassifier
import spacy

In [0]:
spacy_nlp = spacy.load('fr_core_news_sm')
spacy_english = spacy.load('en_core_web_sm')
stopwords = nltk.corpus.stopwords.words('french') #+ nltk.corpus.stopwords.words('english')

In [0]:
def normalize_accent(string):
    string = string.replace('á', 'a')
    string = string.replace('â', 'a')

    string = string.replace('é', 'e')
    string = string.replace('è', 'e')
    string = string.replace('ê', 'e')
    string = string.replace('ë', 'e')

    string = string.replace('î', 'i')
    string = string.replace('ï', 'i')

    string = string.replace('ö', 'o')
    string = string.replace('ô', 'o')
    string = string.replace('ò', 'o')
    string = string.replace('ó', 'o')

    string = string.replace('ù', 'u')
    string = string.replace('û', 'u')
    string = string.replace('ü', 'u')

    string = string.replace('ç', 'c')
    
    return string
def raw_to_tokens(raw_string, spacy_nlp):
    # Write code for lower-casing
    string = raw_string.lower()
    
    # Write code to normalize the accents
    string = normalize_accent(string)
    string = unidecode.unidecode(string)
        
    # Write code to tokenize
    spacy_tokens = spacy_nlp(string)
        
    # Write code to remove punctuation tokens and create string tokens
    string_tokens = [token.lemma_ for token in spacy_tokens if not token.is_punct if not token.is_stop]

    #removing stopwords and non-alphabets
    string_stem = [word for word in string_tokens if (word not in stopwords and not word.isalnum())]  
    
    # Write code to join the tokens back into a single string
    clean_string = " ".join(string_tokens)
    
    return clean_string

In [0]:
s= "Mini Wifi 720p CamÃ©ra Drone Rc Quadcopter 24 Ghz 4ch 6-Axis Gyro 3d Ufo Fpv Rc _1975 @Cocoworld-GÃ©nÃ©rique"
raw_to_tokens(s, spacy_nlp)

'mini wifi 720p cama(c)ra drone rc quadcopter 24 ghz 4ch 6-axis gyro 3d ufo fpv rc 1975 @cocoworld ga(c)na(c)rique'

In [0]:
train_df=pd.read_csv("X_train_update.csv")
test_df=pd.read_csv("X_test_update.csv")
train_Y=pd.read_csv("Y_train.csv")

In [0]:
# train_df=train_df.drop(['description','productid','imageid'],axis=1)
# test_df=test_df.drop(['description','productid','imageid'],axis=1) 

In [0]:
train_df.fillna('0', inplace=True)
test_df.fillna('0', inplace=True)
#x_test.fillna('0')
train_df.head()

,Unnamed: 0,designation,description,productid,imageid
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,0,3804725264,1263597046
1,1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,0,436067568,1008141237
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,0,50418756,457047496
4,4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


In [0]:
##################################Extra features extracted from designation column####################################

# Number of words
train_df['wordcnt'] = train_df['designation'].apply(lambda x: len(str(x).split()))
test_df['wordcnt'] = test_df['designation'].apply(lambda x: len(str(x).split()))

# Number of capital letters
train_df['capitalcnt'] = train_df['designation'].apply(lambda x: len([c for c in str(x) if c.isupper()]))
test_df['capitalcnt'] = test_df['designation'].apply(lambda x: len([c for c in str(x) if c.isupper()]))

#Number of special characters
train_df['specialcnt'] = train_df['designation'].str.findall(r'[^a-zA-Z0-9 ]').str.len()
test_df['specialcnt'] = test_df['designation'].str.findall(r'[^a-zA-Z0-9 ]').str.len()

#Number of unique words
train_df['uniquecnt'] = train_df['designation'].apply(lambda x: len(set(str(x).split())))
test_df['uniquecnt'] = test_df['designation'].apply(lambda x: len(set(str(x).split())))

#Numbers of digits
train_df['digitcnt'] = train_df['designation'].apply(lambda x: sum(c.isdigit() for c in x))
test_df['digitcnt'] = test_df['designation'].apply(lambda x: sum(c.isdigit() for c in x))

#Number of characters
train_df['charcnt'] = train_df['designation'].apply(lambda x: len(str(x)))
test_df['charcnt'] = test_df['designation'].apply(lambda x: len(str(x)))

#Number of stopwords
train_df['stopcnt'] = train_df['designation'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords]))
test_df['stopcnt'] = test_df['designation'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords]))

In [0]:
designation=(train_df['designation'])
#desig=x_test['designation']
vec=[]
for string in designation:
    doc=raw_to_tokens(string,spacy_nlp)
    vec.append(doc)

In [0]:
designation=(test_df['designation'])
#desig=x_test['designation']
vec1=[]
for string in designation:
    doc=raw_to_tokens(string,spacy_nlp)
    vec1.append(doc)

In [0]:
tfidf = TfidfVectorizer(min_df = 2)
train_tfidf = tfidf.fit_transform(vec)
test_tfidf = tfidf.transform(vec1)
print("Shape of the TF-IDF Matrix:")
print((train_tfidf.shape))
print((test_tfidf.shape))
#print(test_tfidf.shape)

In [0]:
#write tfidf vector into csv file
scipy.sparse.save_npz("/content/drive/My Drive/MDSBA/ELTA/train_tfidf_all_features.npz", train_tfidf)
#write tfidf vector into csv file
scipy.sparse.save_npz("/content/drive/My Drive/MDSBA/ELTA/test_tfidf_all_features.npz", test_tfidf)

In [0]:
#loading tfidf matrix into variable
train_tfidf = scipy.sparse.load_npz("train_tfidf_all_features.npz")
test_tfidf = scipy.sparse.load_npz("test_tfidf_all_features.npz")

In [0]:
del train_Y['Unnamed: 0']
train_Y.head()

,prdtypecode
0,10
1,2280
2,50
3,1280
4,2705


In [0]:
# train_df.drop(['Unnamed: 0', 'designation'], axis=1, inplace=True)
# test_df.drop(['Unnamed: 0', 'designation'], axis=1, inplace=True)
# train_df.dtypes, test_df.dtypes
# train_df_arr = train_df.to_numpy()
# test_df_arr = test_df.to_numpy()
# train_df_arr.shape, test_df_arr.shape, train_tfidf.shape, test_tfidf.shape
# train_df_arr = train_df.to_numpy()
# test_df_arr = test_df.to_numpy()
# train_df_arr = scipy.sparse.csr_matrix(train_df.values)
# test_df_arr = scipy.sparse.csr_matrix(test_df.values)
# train_df_arr.shape, test_df_arr.shape, train_tfidf.shape, test_tfidf.shape


# train_all = scipy.sparse.hstack([train_tfidf,train_df_arr])
# test_all = scipy.sparse.hstack([test_tfidf,test_df_arr])
# print(train_all.shape)
# print(test_all.shape)
# train_tfidf = train_tfidf.toarray()
# test_tfidf = test_tfidf.toarray()


# train_all = np.concatenate((train_tfidf,train_df_arr), axis=1)
# test_all = np.concatenate((test_tfidf,test_df_arr), axis=1)
# print(train_all.shape)
# print(test_all.shape)

In [0]:
train_x, test_x, train_y, test_y = train_test_split(train_tfidf, train_Y.values.ravel(), test_size=0.3, random_state=42)

# Bagging

In [0]:
clf = RandomForestClassifier(n_estimators=300)
clf.fit(train_x,train_y)
y_pred_rf = clf.predict(test_x)
print(accuracy_score(y_pred_rf, test_y))
score_rf = f1_score(test_y, y_pred_rf, average = 'weighted')
print(score_rf)

0.7794700686947988
0.7794146341233097


In [0]:
clf = ExtraTreesClassifier(n_estimators=200)
clf.fit(train_x,train_y)
y_pred_et = clf.predict(test_x)
print(accuracy_score(y_pred_et, test_y))
score_et = f1_score(test_y, y_pred_et, average = 'weighted')
print(score_et)

0.7881059862610402
0.7869144408076344


# Boosting

In [0]:
clf = XGBClassifier()
clf.fit(train_x,train_y)
y_pred_xgb = clf.predict(test_x)
print(accuracy_score(y_pred_xgb, test_y))
score_xgb = f1_score(test_y, y_pred_xgb, average = 'weighted')
print(score_xgb)

0.7692247301275761
0.7746888942611513


In [0]:
clf = AdaBoostClassifier(n_estimators=300, learning_rate=0.8)
clf.fit(train_x,train_y)
y_pred_ab = clf.predict(test_x)
print(accuracy_score(y_pred_ab, test_y))
score_ab = f1_score(test_y, y_pred_ab, average = 'weighted')
print(score_ab)

0.3820215897939156
0.35040405841300515


In [0]:
clf = GradientBoostingClassifier(n_estimators=200)
clf.fit(train_x,train_y)
y_pred_gb = clf.predict(test_x)
print(accuracy_score(y_pred_gb, test_y))
score_gb = f1_score(test_y, y_pred_gb, average = 'weighted')
print(score_gb)

0.746810598626104
0.7423995374335393


In [0]:
# clf = HistGradientBoostingClassifier()
# clf.fit(train_x_hgbm,train_y_hgbm)
# y_pred_hgb = clf.predict(test_x_hgbm)
# print(accuracy_score(y_pred_hgb, test_y_hgbm))
# score_hgb = f1_score(test_y_hgbm, y_pred_hgb, average = 'weighted')

In [0]:
#CatBoostClassifier
clf = CatBoostClassifier(iterations=1000, 
                           task_type="GPU",
                           devices='0:1')
clf.fit(train_x,train_y)
y_pred_cb = clf.predict(test_x)
print(accuracy_score(y_pred_cb, test_y))
score_cb = f1_score(test_y, y_pred_cb, average = 'weighted')
print(score_cb)

Learning rate set to 0.149347
0:	learn: 2.7944004	total: 1.45s	remaining: 24m 4s
1:	learn: 2.6242708	total: 2.64s	remaining: 21m 58s
2:	learn: 2.5357734	total: 3.8s	remaining: 21m 4s
3:	learn: 2.4613866	total: 5.19s	remaining: 21m 31s
4:	learn: 2.3894587	total: 6.28s	remaining: 20m 49s
5:	learn: 2.3368719	total: 7.39s	remaining: 20m 24s
6:	learn: 2.2964636	total: 8.53s	remaining: 20m 10s
7:	learn: 2.2659318	total: 9.59s	remaining: 19m 48s
8:	learn: 2.2307853	total: 10.7s	remaining: 19m 34s
9:	learn: 2.2073161	total: 11.8s	remaining: 19m 25s
10:	learn: 2.1881944	total: 13s	remaining: 19m 28s
11:	learn: 2.1486920	total: 14.3s	remaining: 19m 38s
12:	learn: 2.1346854	total: 15.4s	remaining: 19m 29s
13:	learn: 2.1132140	total: 16.4s	remaining: 19m 18s
14:	learn: 2.0990561	total: 17.6s	remaining: 19m 16s
15:	learn: 2.0853834	total: 18.8s	remaining: 19m 18s
16:	learn: 2.0665071	total: 19.9s	remaining: 19m 11s
17:	learn: 2.0523100	total: 21s	remaining: 19m 4s
18:	learn: 2.0368489	total: 22s	re

In [0]:
#LightGBM
clf = lgb.LGBMClassifier()
clf.fit(train_x,train_y)
y_pred_lgb = clf.predict(test_x)
print(accuracy_score(y_pred_lgb, test_y))
score_lgb = f1_score(test_y, y_pred_lgb, average = 'weighted')
print(score_lgb)

0.7656918547595682
0.7690653930206779


# Stacking Classifiers

We have run several prediction models(including non-ensemble methods) to see which are the classifiers giving best accuracies so that it can be used in the stacking algorithms.

In [0]:
clf = LogisticRegressionCV(max_iter=500)
clf.fit(train_x,train_y)
y_pred_lr = clf.predict(test_x)
print(accuracy_score(y_pred_lr, test_y))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8035721295387634


In [0]:
clf = DecisionTreeClassifier()
clf.fit(train_x,train_y)
y_pred_kn = clf.predict(test_x)
print(accuracy_score(y_pred_kn, test_y))

0.7171344455348381


In [0]:
clf = LinearSVC(loss='hinge')
clf.fit(train_x,train_y)
y_pred_lsvm = clf.predict(test_x)
print(accuracy_score(y_pred_lsvm, test_y))

0.8065161923454367


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
clf = SVC()
clf.fit(train_x,train_y)
y_pred_svm = clf.predict(test_x)
print(accuracy_score(y_pred_svm, test_y))

0.8


In [0]:
#stacking - 1
base_learners = [
                 ('lr', LogisticRegression(max_iter=1000)),
                 ('svr', LinearSVC(random_state=42)),
                 ('rf', RandomForestClassifier(n_estimators=30, random_state=42))
                     
                ]
clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression(max_iter=500))
clf.fit(train_x,train_y)
y_pred_sc1 = clf.predict(test_x)
print(accuracy_score(y_pred_sc1, test_y))
score_sc1 = f1_score(test_y, y_pred_sc1, average = 'weighted')
print(score_sc1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8199803729146222
0.8202352295360354


In [0]:
y_pred_sc1_test = clf.predict(test_tfidf)
pred_df = pd.DataFrame(y_pred_sc1_test)
pred_df.to_csv("/content/drive/My Drive/MDSBA/ELTA/stacking1.csv", index=True, index_label='Id')

#### Highest accuracy with Stacking classifier

In [0]:
#stacking - 2 - highest accuracy
base_learners = [
                 ('lr', LogisticRegressionCV(max_iter=1000)),
                 ('svr', LinearSVC(random_state=42)),
                 ('rf', ExtraTreesClassifier(n_estimators=100))
                     
                ]
clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression(max_iter=500))
clf.fit(train_x,train_y)

In [0]:
print(accuracy_score(y_pred_sc2, test_y))
score_sc2 = f1_score(test_y, y_pred_sc2, average = 'weighted')
print(score_sc2)

0.8224926398429833
0.8226002910177455


In [0]:
y_pred_sc2_test = clf.predict(test_tfidf)
pred_df = pd.DataFrame(y_pred_sc2_test)
pred_df.to_csv("stacking2.csv", index=True, index_label='Id')

In [0]:
#stacking - 3
base_learners = [
                 ('lr', LogisticRegression(max_iter=1000)),
                 ('svr', LinearSVC(random_state=42))
                     
                ]
clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression(max_iter=500))
clf.fit(train_x,train_y)
y_pred_sc3 = clf.predict(test_x)
print(accuracy_score(y_pred_sc3, test_y))
score_sc3 = f1_score(test_y, y_pred_sc3, average = 'weighted')
print(score_sc3)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.808243375858685
0.8090231240928568


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
#multilayer stacking - 1
layer_one_estimators = [
                        ('svc', LinearSVC()),
                        ('rf_1', RandomForestClassifier(n_estimators=100, random_state=42))            
                       ]
layer_two_estimators = [
                        ('lr', LogisticRegression(max_iter=1000)),
                        ('rf_2', RandomForestClassifier(n_estimators=100, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression(max_iter = 500))
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)
clf.fit(train_x,train_y)
y_pred_msc1 = clf.predict(test_x)
print(accuracy_score(y_pred_msc1, test_y))
score_msc1 = f1_score(test_y, y_pred_msc1, average = 'weighted')
print(score_msc1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8195878312070658
0.8209223857475714


In [0]:
y_pred_msc1_test = clf.predict(test_tfidf)
pred_df = pd.DataFrame(y_pred_msc1_test)
pred_df.to_csv("/content/drive/My Drive/MDSBA/ELTA/stack1_rf.csv", index=True, index_label='Id')

In [0]:
#multilayer stacking - 2
layer_one_estimators = [
                        ('lr',LogisticRegression(max_iter=1000)),
                        ('rf_1', ExtraTreesClassifier(n_estimators=100, random_state=42))            
                       ]
layer_two_estimators = [
                        ('svc', LinearSVC(loss='hinge')),
                        ('rf_2', ExtraTreesClassifier(n_estimators=100, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression(max_iter = 500))
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)
clf.fit(train_x,train_y)
y_pred_msc2 = clf.predict(test_x)
print(accuracy_score(y_pred_msc2, test_y))
score_msc2 = f1_score(test_y, y_pred_msc2, average = 'weighted')
print(score_msc2)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8065554465161924
0.8075034564481208


# Voting Classifier

In [0]:
#Voting classifier with hard voting
clf1 = LogisticRegression(max_iter=1000)
clf2 = LinearSVC(loss='hinge')
clf3 = ExtraTreesClassifier(n_estimators=200, random_state=42)
eclf = VotingClassifier(estimators=[('lr', clf1), ('svc', clf2), ('rf', clf3)],voting='hard')
clf1 = clf1.fit(train_x,train_y)
clf2 = clf2.fit(train_x,train_y)
clf3 = clf3.fit(train_x,train_y)
eclf = eclf.fit(train_x,train_y)
y_pred_vc1 = eclf.predict(test_x)
print(accuracy_score(y_pred_vc1 , test_y))
score_vc1  = f1_score(test_y, y_pred_vc1 , average = 'weighted')
print(score_vc1)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8062414131501472
0.806192830486673


In [0]:
#voting classifier with soft voting
clf1 = LogisticRegression(max_iter=1000)
clf2 = SVC(kernel='rbf', probability=True)
clf3 = ExtraTreesClassifier(n_estimators=100, random_state=42)
eclf = VotingClassifier(estimators=[('lr', clf1), ('svc', clf2), ('rf', clf3)],voting='soft', weights=[2, 2, 1])
clf1 = clf1.fit(train_x,train_y)
clf2 = clf2.fit(train_x,train_y)
clf3 = clf3.fit(train_x,train_y)
eclf = eclf.fit(train_x,train_y)
y_pred_vc2 = eclf.predict(test_x)
print(accuracy_score(y_pred_vc2 , test_y))
score_vc2  = f1_score(test_y, y_pred_vc2 , average = 'weighted')
print(score_vc2)

0.813189401373896
0.8138272066592334


In [0]:
#hyperparameter grid search for randomforest
n_estimators = [30, 50, 100, 200, 300]
min_samples_split = [2,4,6]
min_samples_leaf = [1, 3, 6]

from sklearn.model_selection import GridSearchCV

hyperF = dict(n_estimators = n_estimators, 
              min_samples_split = min_samples_split,
             min_samples_leaf = min_samples_leaf)
clf = GridSearchCV(RandomForestClassifier(), hyperF, cv = 3, verbose = 1,
                      n_jobs = -1, scoring='accuracy')

clf.fit(train_x,train_y)
clf_best = clf.best_estimator_
y_pred_rfh = clf_best.predict(test_x)
print(accuracy_score(y_pred_rfh, test_y))
score_rfh = f1_score(test_y, y_pred_rfh, average = 'weighted')

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  7.3min
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 32.2min finished


0.7786457311089303


In [0]:
clf.best_params_

{'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 200}

In [0]:
#hyperparameter grid search for extratreeclassifier
n_estimators = [30, 50, 100, 200, 300]
min_samples_split = [2,4,6]
min_samples_leaf = [1, 3, 5, 6]

hyperF = dict(n_estimators = n_estimators,  
              min_samples_split = min_samples_split,
             min_samples_leaf = min_samples_leaf)
clf = GridSearchCV(ExtraTreesClassifier(), hyperF, cv = 3, verbose = 1,
                      n_jobs = -1, scoring='accuracy')

clf.fit(train_x,train_y)
clf_best = clf.best_estimator_
y_pred_eth = clf_best.predict(test_x)
print(accuracy_score(y_pred_eth, test_y))
score_eth = f1_score(test_y, y_pred_eth, average = 'weighted')

In [0]:
clf.best_params_

In [0]:
#hyperparameter grid search for lightgbm
boosting_type = ['gbdt', 'dart', 'goss']
learning_rate = [0.01, 0.05, 0.1]
hyperF = dict(boosting_type = boosting_type,  
              learning_rate=learning_rate)
clf = GridSearchCV(lgb.LGBMClassifier(), hyperF, cv = 3, verbose = 1,
                      n_jobs = -1, scoring='accuracy')

clf.fit(train_x,train_y)
clf_best = clf.best_estimator_
y_pred_lgbh = clf_best.predict(test_x)
print(accuracy_score(y_pred_lgbh, test_y))
score_lgbh = f1_score(test_y, y_pred_lgbh, average = 'weighted')

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed: 12.9min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 14.7min finished


0.7656918547595682


In [0]:
clf.best_params_

{'boosting_type': 'gbdt', 'learning_rate': 0.1}

# Image classification jusing neural networks

In [0]:
# ##################################code used to resize images########################################
# def resize(path, path1):
#     for item in os.listdir(path):
#         if os.path.isfile(path+item):
#             im = Image.open(path+item)
#             f, e = os.path.splitext(path1+item)
#             imResize = im.resize((64,64), Image.ANTIALIAS)
#             imResize.save(f + ' resized.jpg', 'JPEG', quality=90)
# path = 'C:/Users/oshee/Documents/MDSBA/T2/ELTA/images/images/image_train/'
# path1 = 'C:/Users/oshee/Documents/MDSBA/T2/ELTA/images/images/image_train_64/'
# resize(path, path1)
# path = 'C:/Users/oshee/Documents/MDSBA/T2/ELTA/images/images/image_test/'
# path1 = 'C:/Users/oshee/Documents/MDSBA/T2/ELTA/images/images/image_test_64/'
# resize(path, path1)

In [0]:
# ##################################code to unzip the zipfiles containing resized images########################################
# zip_ref1 = zipfile.ZipFile("/content/drive/My Drive/MDSBA/ELTA/image_train_new.zip", 'r')
# zip_ref1.extractall("/content/drive/My Drive/MDSBA/ELTA/")
# zip_ref1.close()
# zip_ref2 = zipfile.ZipFile("/content/drive/My Drive/MDSBA/ELTA/image_test_new.zip", 'r')
# zip_ref2.extractall("/content/drive/My Drive/MDSBA/ELTA/")
# zip_ref2.close()

In [0]:
path = "/content/drive/My Drive/MDSBA/ELTA/"  
train_path = path + 'image_train_new/'
df = pd.DataFrame({'path': glob(os.path.join(train_path,'*.jpg'))})
df['imageid']=df['path'].apply(lambda x: x.split('/ELTA/image_train_new/image_')[1].split('_product')[0])
df['productid']=df['path'].apply(lambda x: x.split('_product_')[1].split(' resized')[0])

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/MDSBA/ELTA/X_train_update.csv')
train_y = pd.read_csv('/content/drive/My Drive/MDSBA/ELTA/Y_train.csv')
train_df['productid'] = train_df['productid'].astype(str)
train_df['imageid'] = train_df['imageid'].astype(str)
train_df = train_df.rename(columns={'Unnamed: 0': 'id'})
train_y = train_y.rename(columns={'Unnamed: 0': 'id'})

In [0]:
df = pd.merge(df, train_df, how='inner', on=['productid','imageid'])
df = pd.merge(df, train_y, how='inner', on='id')
print(df.shape)
print(train_df.shape)

(84916, 7)
(84916, 5)


In [0]:
prd_list = list(df.prdtypecode.unique())
map_dict = {}
for i, id in enumerate(prd_list):
    map_dict[id] = i
df['new_label'] = df['prdtypecode'].apply(lambda x: map_dict[x])
print(df.head())

                                                path  ... new_label
0  /content/drive/My Drive/MDSBA/ELTA/image_train...  ...         0
1  /content/drive/My Drive/MDSBA/ELTA/image_train...  ...         1
2  /content/drive/My Drive/MDSBA/ELTA/image_train...  ...         2
3  /content/drive/My Drive/MDSBA/ELTA/image_train...  ...         3
4  /content/drive/My Drive/MDSBA/ELTA/image_train...  ...         3

[5 rows x 8 columns]


In [0]:
counter = 0
def load_data(N,df):
    """ This functions loads N images using the data df
    """
    X = np.zeros([N,32,32,3]) 
    y = np.squeeze(df.as_matrix(columns=['new_label']))[0:N]
    for i, row in tqdm_notebook(df.iterrows(), total=N):
        if i == N:
            break
        X[i] = cv2.imread(row['path'])
          
    return X,y

In [0]:
N=20000
gc.collect()
X,y = load_data(N=N,df=df) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [0]:
gc.collect()
X_train_tensor = torch.tensor(X, dtype=torch.long)
Y_train_tensor = torch.tensor(y, dtype=torch.long)
full_dataset = torch.utils.data.TensorDataset(X_train_tensor, Y_train_tensor)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=32,
                                           shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=32,
                                           shuffle=False)

In [0]:
# The model has been fine tuned by performing dropout and batch normalization
# *****START CODE
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        #self.drop1 = nn.Dropout(p=0.05)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(512, 512, 3, padding=1)
        self.fc1 = nn.Linear(512 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        # self.drop2 = nn.Dropout(p=0.05)
        self.fc3 = nn.Linear(512, 27)

        
        
        
    def forward(self, x):
        x = x.view(-1, 3, 32, 32) 
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.relu(self.bn2(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        #x = self.drop1(x)
        x = self.relu(self.bn3(self.conv5(x)))
        x = self.pool(self.relu(self.conv6(x)))
        x = x.view(-1, 512 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        # x = self.drop2(x)
        x = self.fc3(x)
        return x


# *****END CODE

In [0]:
# *****START CODE
model = ConvNet()
device = torch.device('cuda:0')
model = model.cuda()

#the best optimizer was found to be adams optimizer, the best loss function was found to be cross entrop loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
epochs = 10
# *****END CODE

In [0]:
## Train the model and validate it after each epoch.
## Provide the train-val loss graph.
# *****START CODE
loss_train = []
loss_eval = []

for epoch in tqdm_notebook(range(epochs)):  # loop over the dataset multiple times

    #training
    training_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.type(torch.FloatTensor)
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        #print(outputs.shape)
        #print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        training_loss += loss.item() #* inputs.size(0)
        # if i % 200 == 199:    # print every 200 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, training_loss / 200))
    loss_train.append(training_loss)
    #validation
    eval_loss = 0.0
    correct = 0.0
    total = 0
    model.eval()
    for i, data in enumerate(val_dataloader, 0):
        images, label = data
        images = images.type(torch.FloatTensor)
        images, label = images.to(device), label.to(device)
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        correct += (predicted == label).sum().item()
        total += label.size(0)
        loss = criterion(output, label)
        eval_loss += loss.item() 
        # if i % 200 == 199:    # print every 200 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, eval_loss / 200))
    loss_eval.append(eval_loss)
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
print('Finished Training')